# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Import
    from collections import Counter
    
    # Extract words from text and sort them
    words_count = Counter(text)
    vocab = sorted(words_count, key=str.lower)
    
    # Create lookup tables (dicts)
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = {i: word for i, word in enumerate(vocab)}
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and 
    the value is the token
    """
    # Define keys and values
    keys = ['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n']
    values = ['||period||', '||comma||', '||quotation_mark||', 
              '||semicolon||', '||exclamation_mark||', 
              '||question_mark||', '||left_parentheses||', 
              '||right_parentheses||', '||dash||', '||return||']
    
    # Create dict
    punctuation_dict = {key: value for key, value in zip(keys, values)}
    return punctuation_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.9.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # Create placeholders
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return (inputs, targets, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # Build the LSTM cell
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell outputs (keep_prob = 0.5)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.5)
        
        return drop
    
    # Stack two LSTM layers for deep learning
    num_layers = 2
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) 
                                        for _ in range(num_layers)])
    
    # Initialize cell state
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), 
                                name='initial_state')
    
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # Create embedding
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), 
                                              -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # Create RNN
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, 
                                             dtype=tf.float32)
    
    # Define final state
    final_state = tf.identity(final_state, name='final_state')
    
    return (outputs, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # Apply embedding
    embed = get_embed(input_data, vocab_size, embed_dim)
    
    # Build RNN
    outputs, final_state = build_rnn(cell, embed)
    
    # Fully connected layer with linear activation
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, 
                                               weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                               activation_fn=None)
    
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # Calculate number of elements per batch and number of batches
    per_batch = batch_size * seq_length
    n_batches = len(int_text) // per_batch
    
    # Limit size of arrays in order to have only full batches
    x = np.array(int_text[:(n_batches * per_batch)])
    y = np.array(int_text[1:(n_batches * per_batch + 1)])
    
    # Define last target value as the first input value 
    y[-1] = x[0]
    
    # Reshape both arrays
    x = x.reshape(batch_size, -1)
    y = y.reshape(batch_size, -1)
    
    # Split arrays
    x = np.split(x, n_batches, 1)
    y = np.split(y, n_batches, 1)
    
    batches = np.array(list(zip(x, y)))
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [25]:
### int_text = 69100


# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 69
# RNN Size
rnn_size = 1024
# Embedding Dimension Size
embed_dim = 3000
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.00001
# Show stats for every n number of batches
show_every_n_batches = 25

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/100   train_loss = 8.841
Epoch   0 Batch   25/100   train_loss = 8.698
Epoch   0 Batch   50/100   train_loss = 8.457
Epoch   0 Batch   75/100   train_loss = 8.379
Epoch   1 Batch    0/100   train_loss = 8.039
Epoch   1 Batch   25/100   train_loss = 7.683
Epoch   1 Batch   50/100   train_loss = 7.412
Epoch   1 Batch   75/100   train_loss = 7.654
Epoch   2 Batch    0/100   train_loss = 7.417
Epoch   2 Batch   25/100   train_loss = 7.173
Epoch   2 Batch   50/100   train_loss = 6.939
Epoch   2 Batch   75/100   train_loss = 7.217
Epoch   3 Batch    0/100   train_loss = 7.136
Epoch   3 Batch   25/100   train_loss = 6.873
Epoch   3 Batch   50/100   train_loss = 6.608
Epoch   3 Batch   75/100   train_loss = 6.989
Epoch   4 Batch    0/100   train_loss = 6.861
Epoch   4 Batch   25/100   train_loss = 6.654
Epoch   4 Batch   50/100   train_loss = 6.423
Epoch   4 Batch   75/100   train_loss = 6.776
Epoch   5 Batch    0/100   train_loss = 6.744
Epoch   5 Batch   25/100   train_l

Epoch  44 Batch   75/100   train_loss = 5.302
Epoch  45 Batch    0/100   train_loss = 5.255
Epoch  45 Batch   25/100   train_loss = 5.130
Epoch  45 Batch   50/100   train_loss = 4.936
Epoch  45 Batch   75/100   train_loss = 5.312
Epoch  46 Batch    0/100   train_loss = 5.190
Epoch  46 Batch   25/100   train_loss = 5.075
Epoch  46 Batch   50/100   train_loss = 4.973
Epoch  46 Batch   75/100   train_loss = 5.291
Epoch  47 Batch    0/100   train_loss = 5.232
Epoch  47 Batch   25/100   train_loss = 5.101
Epoch  47 Batch   50/100   train_loss = 4.915
Epoch  47 Batch   75/100   train_loss = 5.247
Epoch  48 Batch    0/100   train_loss = 5.138
Epoch  48 Batch   25/100   train_loss = 5.013
Epoch  48 Batch   50/100   train_loss = 4.887
Epoch  48 Batch   75/100   train_loss = 5.191
Epoch  49 Batch    0/100   train_loss = 5.125
Epoch  49 Batch   25/100   train_loss = 5.020
Epoch  49 Batch   50/100   train_loss = 4.884
Epoch  49 Batch   75/100   train_loss = 5.191
Epoch  50 Batch    0/100   train_l

Epoch  89 Batch   50/100   train_loss = 4.272
Epoch  89 Batch   75/100   train_loss = 4.459
Epoch  90 Batch    0/100   train_loss = 4.337
Epoch  90 Batch   25/100   train_loss = 4.415
Epoch  90 Batch   50/100   train_loss = 4.206
Epoch  90 Batch   75/100   train_loss = 4.443
Epoch  91 Batch    0/100   train_loss = 4.305
Epoch  91 Batch   25/100   train_loss = 4.329
Epoch  91 Batch   50/100   train_loss = 4.258
Epoch  91 Batch   75/100   train_loss = 4.437
Epoch  92 Batch    0/100   train_loss = 4.336
Epoch  92 Batch   25/100   train_loss = 4.418
Epoch  92 Batch   50/100   train_loss = 4.175
Epoch  92 Batch   75/100   train_loss = 4.398
Epoch  93 Batch    0/100   train_loss = 4.275
Epoch  93 Batch   25/100   train_loss = 4.345
Epoch  93 Batch   50/100   train_loss = 4.152
Epoch  93 Batch   75/100   train_loss = 4.375
Epoch  94 Batch    0/100   train_loss = 4.295
Epoch  94 Batch   25/100   train_loss = 4.305
Epoch  94 Batch   50/100   train_loss = 4.169
Epoch  94 Batch   75/100   train_l

Epoch 134 Batch   25/100   train_loss = 3.769
Epoch 134 Batch   50/100   train_loss = 3.601
Epoch 134 Batch   75/100   train_loss = 3.823
Epoch 135 Batch    0/100   train_loss = 3.581
Epoch 135 Batch   25/100   train_loss = 3.776
Epoch 135 Batch   50/100   train_loss = 3.531
Epoch 135 Batch   75/100   train_loss = 3.756
Epoch 136 Batch    0/100   train_loss = 3.636
Epoch 136 Batch   25/100   train_loss = 3.778
Epoch 136 Batch   50/100   train_loss = 3.622
Epoch 136 Batch   75/100   train_loss = 3.711
Epoch 137 Batch    0/100   train_loss = 3.611
Epoch 137 Batch   25/100   train_loss = 3.686
Epoch 137 Batch   50/100   train_loss = 3.600
Epoch 137 Batch   75/100   train_loss = 3.774
Epoch 138 Batch    0/100   train_loss = 3.560
Epoch 138 Batch   25/100   train_loss = 3.783
Epoch 138 Batch   50/100   train_loss = 3.523
Epoch 138 Batch   75/100   train_loss = 3.742
Epoch 139 Batch    0/100   train_loss = 3.518
Epoch 139 Batch   25/100   train_loss = 3.645
Epoch 139 Batch   50/100   train_l

Epoch 179 Batch    0/100   train_loss = 2.982
Epoch 179 Batch   25/100   train_loss = 3.193
Epoch 179 Batch   50/100   train_loss = 3.107
Epoch 179 Batch   75/100   train_loss = 3.159
Epoch 180 Batch    0/100   train_loss = 3.069
Epoch 180 Batch   25/100   train_loss = 3.172
Epoch 180 Batch   50/100   train_loss = 3.125
Epoch 180 Batch   75/100   train_loss = 3.147
Epoch 181 Batch    0/100   train_loss = 3.079
Epoch 181 Batch   25/100   train_loss = 3.189
Epoch 181 Batch   50/100   train_loss = 3.082
Epoch 181 Batch   75/100   train_loss = 3.121
Epoch 182 Batch    0/100   train_loss = 3.009
Epoch 182 Batch   25/100   train_loss = 3.160
Epoch 182 Batch   50/100   train_loss = 2.978
Epoch 182 Batch   75/100   train_loss = 3.135
Epoch 183 Batch    0/100   train_loss = 2.997
Epoch 183 Batch   25/100   train_loss = 3.143
Epoch 183 Batch   50/100   train_loss = 2.999
Epoch 183 Batch   75/100   train_loss = 3.207
Epoch 184 Batch    0/100   train_loss = 2.996
Epoch 184 Batch   25/100   train_l

...

Epoch 805 Batch   50/100   train_loss = 0.654
Epoch 805 Batch   75/100   train_loss = 0.717
Epoch 806 Batch    0/100   train_loss = 0.615
Epoch 806 Batch   25/100   train_loss = 0.740
Epoch 806 Batch   50/100   train_loss = 0.676
Epoch 806 Batch   75/100   train_loss = 0.736
Epoch 807 Batch    0/100   train_loss = 0.640
Epoch 807 Batch   25/100   train_loss = 0.744
Epoch 807 Batch   50/100   train_loss = 0.678
Epoch 807 Batch   75/100   train_loss = 0.715
Epoch 808 Batch    0/100   train_loss = 0.642
Epoch 808 Batch   25/100   train_loss = 0.777
Epoch 808 Batch   50/100   train_loss = 0.678
Epoch 808 Batch   75/100   train_loss = 0.728
Epoch 809 Batch    0/100   train_loss = 0.596
Epoch 809 Batch   25/100   train_loss = 0.738
Epoch 809 Batch   50/100   train_loss = 0.631
Epoch 809 Batch   75/100   train_loss = 0.737
Epoch 810 Batch    0/100   train_loss = 0.643
Epoch 810 Batch   25/100   train_loss = 0.751
Epoch 810 Batch   50/100   train_loss = 0.693
Epoch 810 Batch   75/100   train_l

Epoch 850 Batch   25/100   train_loss = 0.715
Epoch 850 Batch   50/100   train_loss = 0.629
Epoch 850 Batch   75/100   train_loss = 0.685
Epoch 851 Batch    0/100   train_loss = 0.623
Epoch 851 Batch   25/100   train_loss = 0.741
Epoch 851 Batch   50/100   train_loss = 0.639
Epoch 851 Batch   75/100   train_loss = 0.682
Epoch 852 Batch    0/100   train_loss = 0.602
Epoch 852 Batch   25/100   train_loss = 0.709
Epoch 852 Batch   50/100   train_loss = 0.660
Epoch 852 Batch   75/100   train_loss = 0.696
Epoch 853 Batch    0/100   train_loss = 0.589
Epoch 853 Batch   25/100   train_loss = 0.728
Epoch 853 Batch   50/100   train_loss = 0.640
Epoch 853 Batch   75/100   train_loss = 0.688
Epoch 854 Batch    0/100   train_loss = 0.561
Epoch 854 Batch   25/100   train_loss = 0.715
Epoch 854 Batch   50/100   train_loss = 0.646
Epoch 854 Batch   75/100   train_loss = 0.645
Epoch 855 Batch    0/100   train_loss = 0.575
Epoch 855 Batch   25/100   train_loss = 0.702
Epoch 855 Batch   50/100   train_l

Epoch 895 Batch    0/100   train_loss = 0.546
Epoch 895 Batch   25/100   train_loss = 0.700
Epoch 895 Batch   50/100   train_loss = 0.613
Epoch 895 Batch   75/100   train_loss = 0.634
Epoch 896 Batch    0/100   train_loss = 0.557
Epoch 896 Batch   25/100   train_loss = 0.687
Epoch 896 Batch   50/100   train_loss = 0.617
Epoch 896 Batch   75/100   train_loss = 0.634
Epoch 897 Batch    0/100   train_loss = 0.566
Epoch 897 Batch   25/100   train_loss = 0.727
Epoch 897 Batch   50/100   train_loss = 0.618
Epoch 897 Batch   75/100   train_loss = 0.662
Epoch 898 Batch    0/100   train_loss = 0.543
Epoch 898 Batch   25/100   train_loss = 0.682
Epoch 898 Batch   50/100   train_loss = 0.600
Epoch 898 Batch   75/100   train_loss = 0.640
Epoch 899 Batch    0/100   train_loss = 0.546
Epoch 899 Batch   25/100   train_loss = 0.694
Epoch 899 Batch   50/100   train_loss = 0.608
Epoch 899 Batch   75/100   train_loss = 0.633
Epoch 900 Batch    0/100   train_loss = 0.565
Epoch 900 Batch   25/100   train_l

Epoch 939 Batch   75/100   train_loss = 0.631
Epoch 940 Batch    0/100   train_loss = 0.519
Epoch 940 Batch   25/100   train_loss = 0.654
Epoch 940 Batch   50/100   train_loss = 0.587
Epoch 940 Batch   75/100   train_loss = 0.610
Epoch 941 Batch    0/100   train_loss = 0.580
Epoch 941 Batch   25/100   train_loss = 0.653
Epoch 941 Batch   50/100   train_loss = 0.558
Epoch 941 Batch   75/100   train_loss = 0.621
Epoch 942 Batch    0/100   train_loss = 0.536
Epoch 942 Batch   25/100   train_loss = 0.680
Epoch 942 Batch   50/100   train_loss = 0.568
Epoch 942 Batch   75/100   train_loss = 0.608
Epoch 943 Batch    0/100   train_loss = 0.552
Epoch 943 Batch   25/100   train_loss = 0.670
Epoch 943 Batch   50/100   train_loss = 0.594
Epoch 943 Batch   75/100   train_loss = 0.623
Epoch 944 Batch    0/100   train_loss = 0.528
Epoch 944 Batch   25/100   train_loss = 0.647
Epoch 944 Batch   50/100   train_loss = 0.596
Epoch 944 Batch   75/100   train_loss = 0.584
Epoch 945 Batch    0/100   train_l

Epoch 984 Batch   50/100   train_loss = 0.572
Epoch 984 Batch   75/100   train_loss = 0.577
Epoch 985 Batch    0/100   train_loss = 0.497
Epoch 985 Batch   25/100   train_loss = 0.665
Epoch 985 Batch   50/100   train_loss = 0.555
Epoch 985 Batch   75/100   train_loss = 0.594
Epoch 986 Batch    0/100   train_loss = 0.493
Epoch 986 Batch   25/100   train_loss = 0.640
Epoch 986 Batch   50/100   train_loss = 0.558
Epoch 986 Batch   75/100   train_loss = 0.611
Epoch 987 Batch    0/100   train_loss = 0.520
Epoch 987 Batch   25/100   train_loss = 0.630
Epoch 987 Batch   50/100   train_loss = 0.563
Epoch 987 Batch   75/100   train_loss = 0.608
Epoch 988 Batch    0/100   train_loss = 0.499
Epoch 988 Batch   25/100   train_loss = 0.661
Epoch 988 Batch   50/100   train_loss = 0.575
Epoch 988 Batch   75/100   train_loss = 0.610
Epoch 989 Batch    0/100   train_loss = 0.523
Epoch 989 Batch   25/100   train_loss = 0.612
Epoch 989 Batch   50/100   train_loss = 0.604
Epoch 989 Batch   75/100   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [30]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # Get tensors from loaded_graph
    input_tensor = loaded_graph.get_tensor_by_name(name="input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    return (input_tensor, initial_state_tensor, final_state, probs_tensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [31]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # Get highest value in probabilities and use it as index to find word in dict
    highest_prob_index = np.argmax(probabilities)
    next_word = int_to_vocab[highest_prob_index]
    return next_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [32]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(sobs) no! no! help me!
moe_szyslak: wait a minute. if(mad) homer, you keep your father, my problems off the and i loved two.
homer_simpson: i, uh, but i'm not a.
moe_szyslak: hey, you just been here. no answer!
moe_szyslak:(, pleading) ah, well, my be... just a sec... ivana tinkle. ivana tinkle.
captain: har, har, har. you mean.
homer_simpson:(to homer) i'm sorry for. but you mean, like i should not time.
moe_szyslak: oh, i don't know. what would really? think about--
lenny_leonard: i could that?(scanning)
homer_simpson: well,, this is... i miss my...
homer_simpson: i'm not, i.(to learn) the new guy..
moe_szyslak: ooh. but where the would i tell you that is your story i ever around.
bart_simpson: i'm thankful to i before not


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.